In [1]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
import time
import requests

In [90]:
#크롬 웹드라이버 불러오기
driver = webdriver.Chrome()
res = driver.get('https://m.place.naver.com/place/list?query=제주도관광지&level=top')
driver.implicitly_wait(20)


In [91]:
#2차 크롤링을 위한 bs4 셋팅
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])

session.mount('http://', HTTPAdapter(max_retries=retries))


In [83]:
#body부분을 잡기 위해 쓸데없이 버튼을 클릭해줌
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[1]/a').click()

#검색결과가 모두 보이지 않기 때문에 page down을 눌러 끝까지 펼쳐준다.
for scroll in range(0,30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)


In [84]:
from bs4 import BeautifulSoup
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')
soup = bs.select_one('div.YluNG')
naver_info = soup.select('li.VLTHu')



In [35]:
from tqdm import tqdm
test = bs.find_all('a', class_='P7gyV')
test_res = []
test_str = []

for x in tqdm(test):
    str_x = str(x).split('/')[4].split('?')[0]
    print(str_x)
    
    if not str_x[0].isdigit():
        continue
    
    # Check if str_x contains any English characters
    if any(char.isalpha() for char in str_x):
        continue
    
    if str_x not in test_res:
        test_res.append(str_x)



100%|██████████| 208/208 [00:00<00:00, 8922.04it/s]

_5LookBsMSiwC9AgAcOPg4R43Kc3p7psBPICqj8_84geEgSmVW5wCSIx8wWoxikPpj7nRBaZFK7fyhoOlLL2x0erQQ2NS9o-jpmeignSZuYlKxbJPV5NilKjaY8Ar6xU_UwB8Vmhi0knBUQ5HkL3sTeW_CGxyc3Y1LNPUePJE9MVedY9N6zZplzLvpjROaiQ2dyyAkRnLKFRJhbk0-HhPssA_CoiP48rlYgJnkIkn0c3n4Bwnzydf6bIGsfLfsVlLR6q7LRek_tzeI8OGxCZZo-Q1UWejarh5QaN2RO9Gnc6TD16WCmVrTnNxG5LDfO436SRfoq9h1jYcOjphmbJlGHFPM2EqPl7kljs6jIcOCeQXYMnobolFB833H0B-VFSv6PF2HVreRDBxgH2JBYKRzXvWBAc6n1vMBF-Zxy-X0uLBAcr7S-JXIi_yCmviBFL1osQGjL6PjtpXKEiGtWJRO2lyJb6_YtNqMix8FMwIEfMQpm92YdkOBtFzAg9xkYrWPwyBf7S6hsmM_2ErtoS9URisk3cSyWXU-kkcIXXZAC3TRvzTd-cZkq_OW5CVntc8kGrDoSkNNrVb4P5_ZZmzgcZ1pLSzdNy9EhfM8NPatIecKHke_PSKoxlWWT-1a8slRdjhynk-S0XsolT77StnWWc-L9ukMWdFdgsKcLItt9yfUlRuB9hvBI5R9VW4hs1rpln1A-leuzDn5xrtwEOlw_7PxJFRit6W1zpiZigdbv66nk8Aea_ejnplhI2tAUr97ibQ4vWHDhRZ0BMne905WEyIRU2dP5YTS_OxNDrB8k4-AUL-8kfYAj2ENwTA6bWWWDuGCgwdqLG7sZyq1MVQEjrB8mOFIfy3MfIAeU3Pfs=
_5LookBsMSiwC9AgAcOPg4R43Kc3p7psBPICqj8_84geEgSmVW5wCSIx8wWoxikPpj7nRBaZFK7fyhoOlLL2x0erQQ2NS9o-jpmeignSZuYlKxbJPV5NilKjaY8

In [23]:
test_res.__len__()

100

In [118]:
import pandas as pd
for place_ids in test_res[99:100]:
    driver = webdriver.Chrome()
    url = f"https://m.place.naver.com/place/{place_ids}/review/visitor?entry=ple"
    driver.get(url)

    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(5)
    #driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(4) > div.NSTUp > div > a.fvwqf:nth-child(3)').click()


    driver.implicitly_wait(30)
    try:
        while True:
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > a > svg").click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')
    # keyword for문
    keyword_content = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li:nth-child(1) > div.CsBE9 > span.nWiXa")
    keyword_count = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li:nth-child(1) > div.CsBE9 > span.TwM9q ")

    time.sleep(5)

    try:
        while True:
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.NSTUp > div > a > span").click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')

    # review for문
    review_content = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.ZZ4OK > a > span.zPfVt')
    review_date = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.qM6I7 > div > div._7kR3e > span:nth-child(1) > time')

    # exception handling
    keyword_content = keyword_content.text if keyword_content else ''
    keyword_count = keyword_count.text.split('\n')[-1] if keyword_count else ''
    review_content = review_content.text if review_content else ''
    review_date = review_date .text if review_date else ''
    time.sleep(0.06)

    # append sheet
    print(test_res[0],'/', keyword_content,'/',keyword_count, '/', review_content, '/', review_date )

    
    time.sleep(0.06)





finish
finish
33703960 / "뷰가 좋아요" / 107 / 용두암, 용연다리 갔다가 방문한 찐 점심식사로 넘 만족한 맛집 🥇🌊노도고등어회🌊🥇진짜 여기 남녀노소 누구나 맛있게 먹을수 있는 음식들이 많아 요!! 메인 고등어회랑 딱새우회는 진짜 제가 먹어본 곳들중 제일 쫄깃쫄깃 탱탱 신선했어요 너무 맛있었고 물회도 아주 시원하고 꼭 밥말아드세요!!ㅋㅋ ㅋ 극락가요 두툼한 돔베고기도 팔고 있어서 육식파 물고기파 둘다 싸우지 않고 즐길수있어서 좋았구요 !앞에 오션뷰는 역대급이였습니다 진짜 ... / 23.11.13.월


In [148]:
#이거 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 데이터를 저장할 데이터프레임 만들기
columns_keyword = ['PlaceID', 'KeywordContent', 'KeywordCount']
df_keyword = pd.DataFrame(columns=columns_keyword)

columns_review = ['PlaceID', 'ReviewContent', 'ReviewDate']
df_review = pd.DataFrame(columns=columns_review)

# 예시로 99번째에서 100번째까지의 place_ids에 대해서 데이터 수집
for place_ids in test_res[99:100]:
    driver = webdriver.Chrome()
    url = f"https://m.place.naver.com/place/{place_ids}/review/visitor?entry=ple"
    driver.get(url)

    

    try:
        while True:
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > a ").click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')

    # keyword_content의 수를 구하기 위해 해당 CSS 선택자에 매치되는 요소들을 가져오기
    keyword_content_elements = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(5) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li:nth-child(1) > div.CsBE9 > span.nWiXa")

    # 가져온 요소들의 개수만큼 for 루프 돌리기
    for i in range(0,len(keyword_content_elements)):
        keyword_content = keyword_content_elements[i]
        keyword_count = driver.find_element(By.CSS_SELECTOR, f"#app-root > div > div > div > div:nth-child(5) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li:nth-child({i+1}) > div.CsBE9 > span.TwM9q")
        # exception handling
        keyword_content = keyword_content.text if keyword_content else ''
        keyword_count = keyword_count.text.split('\n')[-1] if keyword_count else ''
        df_keyword = pd.concat([df_keyword, pd.DataFrame([[place_ids, keyword_content, keyword_count]], columns=columns_keyword)], ignore_index=True)
    time.sleep(0.06)


    try:
        while True:
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.NSTUp > div > a > span").click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')
    
    time.sleep(0.06)
    
    review_content_elements = driver.find_elements(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.ZZ4OK > a > span.zPfVt')
    # 리뷰 정보 수집
    for i in range(len(review_content_elements)):
        review_content = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.ZZ4OK > a > span.zPfVt')
        review_date = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.qM6I7 > div > div._7kR3e > span:nth-child(1) > time')
        review_content = review_content.text if review_content else ''
        review_date = review_date .text if review_date else ''
        df_review = pd.concat([df_review, pd.DataFrame([[place_ids, review_content, review_date]], columns=columns_review)], ignore_index=True)
        time.sleep(0.06)

    # 데이터프레임에 추가
    
    



# 데이터프레임 출력
print("Keyword Dataframe:")
print(df_keyword)
print("\nReview Dataframe:")
print(df_review)


finish
finish
Keyword Dataframe:
    PlaceID KeywordContent KeywordCount
0  13491801       "뷰가 좋아요"          107

Review Dataframe:
Empty DataFrame
Columns: [PlaceID, ReviewContent, ReviewDate]
Index: []


In [150]:
driver = webdriver.Chrome()
url = f"https://m.place.naver.com/place/13491801/review/visitor?entry=ple"
driver.get(url)

    

try:
    while True:
        driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > a  ").click()
        time.sleep(0.4)
except Exception as e:
        print('finish')

finish


In [142]:
for i in range(len(keyword_content_elements)):
    print(keyword_content_elements[i].text)

"뷰가 좋아요"
"산책로가 잘 되어있어요"
"사진이 잘 나와요"
"주차하기 편해요"
"볼거리가 많아요"
"조용히 쉬기 좋아요"
"근처에 갈 곳이 많아요"
"가격이 합리적이에요"
"경관이 독특해요"
"편의시설이 잘 되어있어요"
"방문객이 많아요"
"관리가 잘 되어있어요"
"아이와 가기 좋아요"
"대중교통이 편해요"
"반려동물과 가기 좋아요"
"화장실이 깨끗해요"
"붐비지 않아요"


In [128]:
df_keyword

,PlaceID,KeywordContent,KeywordCount
0,13491801,"""볼거리가 많아요""",20


In [80]:

for place_ids in test_res[:2]:
    # 브라우저 열기
    driver = webdriver.Chrome()

    # URL 설정
    url = f"https://m.place.naver.com/place/{place_ids}/home?entry=ple"

    # URL로 이동
    driver.get(url)

    driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(3) > span").click()
    driver.implicitly_wait(30)

    # '더보기' 버튼 계속 클릭
    try:
        while True:
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > a > svg").click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')

    # keyword_content의 수를 구하기 위해 해당 CSS 선택자에 매치되는 요소들을 가져오기
    keyword_content_elements = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li > div.CsBE9 > span.nWiXa")

    # 가져온 요소들의 개수만큼 for 루프 돌리기
    for i in range(len(keyword_content_elements)):
        keyword_content = keyword_content_elements[i]
        keyword_count = driver.find_element(By.CSS_SELECTOR, f"#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li:nth-child({i+1}) > div.CsBE9 > span.TwM9q")
        # exception handling
        keyword_content = keyword_content.text if keyword_content else ''
        keyword_count = keyword_count.text.split('\n')[-1] if keyword_count else ''
        
        print(f"Keyword Content {i+1}: {keyword_content}")
        print(f"Keyword Count {i+1}: {keyword_count}")
        time.sleep(0.06)
    # 리뷰 더보기 버튼
    try:
        while True:
            driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.NSTUp > div > a > span").click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')

    review_content_elements = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.ZZ4OK > a > span.zPfVt')

    # review for문
    for i in range(len(review_content_elements)):
        review_content = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.ZZ4OK > a > span.zPfVt')
        review_date = driver.find_element(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child(1) > div.qM6I7 > div > div._7kR3e > span:nth-child(1) > time')
        review_content = review_content.text if review_content else ''
        review_date = review_date .text if review_date else ''
        time.sleep(0.06)






finish
Keyword Content 1: "볼거리가 많아요"
Keyword Count 1: 4155
Keyword Content 2: "아이와 가기 좋아요"
Keyword Count 2: 4005
Keyword Content 3: "사진이 잘 나와요"
Keyword Count 3: 3816
Keyword Content 4: "주차하기 편해요"
Keyword Count 4: 2126
Keyword Content 5: "체험 프로그램이 다양해요"
Keyword Count 5: 734
Keyword Content 6: "가격이 합리적이에요"
Keyword Count 6: 488
Keyword Content 7: "놀이기구가 다양해요"
Keyword Count 7: 366
Keyword Content 8: "먹거리가 풍부해요"
Keyword Count 8: 197
Keyword Content 9: "규모가 커요"
Keyword Count 9: 102
Keyword Content 10: "시설이 깔끔해요"
Keyword Count 10: 93
Keyword Content 11: "컨셉이 독특해요"
Keyword Count 11: 56
Keyword Content 12: "친절해요"
Keyword Count 12: 52
Keyword Content 13: "대기시간이 짧아요"
Keyword Count 13: 34
Keyword Content 14: "편의시설이 잘 되어있어요"
Keyword Count 14: 20
Keyword Content 15: "유익해요"
Keyword Count 15: 9
Keyword Content 16: "안전하게 관리해요"
Keyword Count 16: 8
Keyword Content 17: "동물 관리가 잘 되어있어요"
Keyword Count 17: 1
Keyword Content 18: "반려동물과 가기 좋아요"
Keyword Count 18: 1
finish


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF7D6C62142+3514994]
	(No symbol) [0x00007FF7D6880CE2]
	(No symbol) [0x00007FF7D67276AA]
	(No symbol) [0x00007FF7D6700AFD]
	(No symbol) [0x00007FF7D679CB1B]
	(No symbol) [0x00007FF7D67B218F]
	(No symbol) [0x00007FF7D6795D93]
	(No symbol) [0x00007FF7D6764BDC]
	(No symbol) [0x00007FF7D6765C64]
	GetHandleVerifier [0x00007FF7D6C8E16B+3695259]
	GetHandleVerifier [0x00007FF7D6CE6737+4057191]
	GetHandleVerifier [0x00007FF7D6CDE4E3+4023827]
	GetHandleVerifier [0x00007FF7D69B04F9+689705]
	(No symbol) [0x00007FF7D688C048]
	(No symbol) [0x00007FF7D6888044]
	(No symbol) [0x00007FF7D68881C9]
	(No symbol) [0x00007FF7D68788C4]
	BaseThreadInitThunk [0x00007FFAE5417344+20]
	RtlUserThreadStart [0x00007FFAE57026B1+33]


In [21]:
#2차 크롤링을 위한 url
url = 'https://m.place.naver.com'
i = 0
place_ids = []
for info in naver_info[:10]:

    tourist_destination = info.select_one('span.YwYLL').text
    sectors = info.select_one('span.YzBgS').text
    link = info.select_one('div.ouxiq').select_one('a').attrs['href']
    time.sleep(0.06)

    #네이버 플레이스로 이동(place ID로 접속)
    N_res = session.get(link, headers=headers)
    N_soup_srch = BeautifulSoup(N_res.content, 'html.parser')
    mart_oldtel = N_soup_srch.select_one('span.yxkiA > a').attrs['href']
    mart_tel = str(re.sub('tel:', '', mart_oldtel)) # 'tel:' 삭제
    pattern = re.compile("/place/([0-9]+)?c")
    place_ids.append(test_res[i])
    #주소는 '공유'에서 파싱
    address = N_soup_srch.select('span.yxkiA > a')[3].attrs['data-line-description']

    print(tourist_destination, '/',sectors,'/',place_ids[i] ,'/', link, '/',  mart_tel, '/', address )
    time.sleep(0.06)
    i+=1


해녀태왁 / 체험여행 / 33703960 / https://tivan.naver.com/d/_5LookBsMSiwC9AgAcOPg4R43Kc3p7psBPICqj8_84geEgSmVW5wCSIx8wWoxikPpj7nRBaZFK7fyhoOlLL2x0erQQ2NS9o-jpmeignSZuYlKxbJPV5NilKjaY8Ar6xU_UwB8Vmhi0knBUQ5HkL3sTeW_CGxyc3Y1LNPUePJE9MVedY9N6zZplzLvpjROaiQ2dyyAkRnLKFRJhbk0-HhPssA_CoiP48rlYgJnkIkn0c3n4Bwnzydf6bIGsfLfsVlLR6q7LRek_tzeI8OGxCZZo-Q1UWejarh5QaN2RO9Gnc6TD16WCmVrTnNxG5LDfO436SRfoq9h1jYcOjphmbJlGHFPM2EqPl7kljs6jIcOCeQXYMnobolFB833H0B-VFSv6PF2HVreRDBxgH2JBYKRzXvWBAc6n1vMBF-Zxy-X0uLBAcr7S-JXIi_yCmviBFL1osQGjL6PjtpXKEiGtWJRO2lyJb6_YtNqMix8FMwIEfMQpm92YdkOBtFzAg9xkYrWPwyBf7S6hsmM_2ErtoS9URisk3cSyWXU-kkcIXXZAC3TRvzTd-cZkq_OW5CVntc8kGrDoSkNNrVb4P5_ZZmzgcZ1pLSzdNy9EhfM8NPatIecKHke_PSKoxlWWT-1a8slRdjhynk-S0XsolT77StnWWc-L9ukMWdFdgsKcLItt9yfUlRuB9hvBI5R9VW4hs1rpln1A-leuzDn5xrtwEOlw_7PxJFRit6W1zpiZigdbv66nk8Aea_ejnplhI2tAUr97ibQ4vWHDhRZ0BMne905WEyIRU2dP5YTS_OxNDrB8k4-AUL-8kfYAj2ENwTA6bWWWDuGCgwdqLG7sZyq1MVQEjrB8mOFIfy3MfIAeU3Pfs=?fu=https%3A%2F%2Fm.place.naver.com%2Fplace%2F1264956495%3Fentry%3Dple%26

In [7]:
import pandas as pd

# 2차 크롤링을 위한 기본 URL
url = 'https://m.place.naver.com'

# 데이터를 담을 빈 리스트 생성
data = {'Tourist Destination': [],
        'Sectors': [],
        'Link': [],
        'Mart Tel': [],
        'Address': []}

for info in naver_info[:10]:
    # 각 정보 수집
    tourist_destination = info.select_one('span.YwYLL').text
    sectors = info.select_one('span.YzBgS').text
    link = info.select_one('div.ouxiq').select_one('a').attrs['href']
    time.sleep(0.06)

    # 네이버 플레이스로 이동(place ID로 접속)
    N_res = session.get(link, headers=headers)
    N_soup_srch = BeautifulSoup(N_res.content, 'html.parser')
    mart_oldtel = N_soup_srch.select_one('span.yxkiA > a').attrs['href']
    mart_tel = str(re.sub('tel:', '', mart_oldtel))  # 'tel:' 삭제
    
    # 주소는 '공유'에서 파싱
    address = N_soup_srch.select('span.yxkiA > a')[3].attrs['data-line-description']

    # 데이터를 딕셔너리에 추가
    data['Tourist Destination'].append(tourist_destination)
    data['Sectors'].append(sectors)
    data['Link'].append(link)
    data['Mart Tel'].append(mart_tel)
    data['Address'].append(address)

    time.sleep(0.06)

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임 출력
print(df)


  Tourist Destination  Sectors  \
0                해녀태왁     체험여행   
1            피규어뮤지엄제주      박물관   
2            헬로키티아일랜드     테마파크   
3              동백포레스트  식물원,수목원   
4           아쿠아플라넷 제주    아쿠아리움   
5               카멜리아힐  식물원,수목원   
6                 쇠소깍     강,하천   
7                새별오름       오름   
8               스누피가든    관람,체험   
9                사계해안  해수욕장,해변   

                                                Link        Mart Tel  \
0  https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pB...    064-782-6347   
1  https://tivan.naver.com/d/kvJ1LB8jzKSK4llyrmdp...  0507-1433-2264   
2  https://m.place.naver.com/place/33703960?entry...    064-792-6114   
3  https://m.place.naver.com/place/1611903913?ent...  0507-1331-2102   
4  https://m.place.naver.com/place/19708047?entry...       1833-7001   
5  https://m.place.naver.com/place/11846866?entry...    064-800-6296   
6  https://m.place.naver.com/place/11707633?entry...               #   
7  https://m.place.naver.com/place/17029023?entry

In [8]:
df

,Tourist Destination,Sectors,Link,Mart Tel,Address
0,해녀태왁,체험여행,https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pB...,064-782-6347,제주 제주시 구좌읍 김송로 94
1,피규어뮤지엄제주,박물관,https://tivan.naver.com/d/kvJ1LB8jzKSK4llyrmdp...,0507-1433-2264,제주 서귀포시 안덕면 한창로 243
2,헬로키티아일랜드,테마파크,https://m.place.naver.com/place/33703960?entry...,064-792-6114,제주 서귀포시 안덕면 한창로 340
3,동백포레스트,"식물원,수목원",https://m.place.naver.com/place/1611903913?ent...,0507-1331-2102,제주 서귀포시 남원읍 생기악로 53-38
4,아쿠아플라넷 제주,아쿠아리움,https://m.place.naver.com/place/19708047?entry...,1833-7001,제주 서귀포시 성산읍 섭지코지로 95 아쿠아플라넷 제주
5,카멜리아힐,"식물원,수목원",https://m.place.naver.com/place/11846866?entry...,064-800-6296,제주 서귀포시 안덕면 병악로 166
6,쇠소깍,"강,하천",https://m.place.naver.com/place/11707633?entry...,#,제주 서귀포시 쇠소깍로 104
7,새별오름,오름,https://m.place.naver.com/place/17029023?entry...,#,제주 제주시 애월읍 봉성리 산59-8
8,스누피가든,"관람,체험",https://m.place.naver.com/place/1875149782?ent...,064-903-1111,제주 제주시 구좌읍 금백조로 930
9,사계해안,"해수욕장,해변",https://m.place.naver.com/place/17083214?entry...,#,제주 서귀포시 안덕면 사계리


In [17]:
place_ids = test_res

In [78]:
test_res.__len__()

4